# JSOC data export

In [ ]:
import os
import drms
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import dates
%matplotlib notebook

Create the download directory, if it does not exist yet.

In [ ]:
out_dir = 'downloads'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

Change the `myemail` address below to your own email address. Note that you need to register you email address on the [JSOC email registration](http://jsoc.stanford.edu/ajax/register_email.html) webpage first.

In [ ]:
myemail = 'name@example.com'
client = drms.Client(email=myemail, verbose=True)

## Download a HMI full-disk image

Export requests are created using the [`drms.Client.export()`](http://drms.readthedocs.io/en/stable/generated/drms.Client.export.html) method. The default protocol is `'as-is'` which is the fastest way to export files from JSOC, but does not include the metadata from the DRMS database in the FITS header. If metadata is required in the FITS headers, the `protocol='fits'` argument must be specified.

In [ ]:
request = client.export('hmi.ic_720s[2015.12.19_00:00_TAI]')
print(request)

In [ ]:
res = request.download(out_dir)

In [ ]:
res

Read the downloaded FITS file:

In [ ]:
fname_ic = res.download[0]
img_data = fits.getdata(fname_ic)

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(img_data/1e3, cmap='gray', origin='lower')
fig.colorbar(im, label='Intensity [kDN/s]')
fig.tight_layout()
fig.show()

In [ ]:
ax.set_xlim(1900, 2200)
ax.set_ylim(1440, 1740)
plt.draw()

Open and inspect the downloaded FITS file:

In [ ]:
f = fits.open(fname_ic)
f.info()

In [ ]:
f[1].header

## Export FITS files with metadata

In order to download FITS files with header data, it is necessary to use the `protocol='fits'` argument when calling [`drms.Client.export()`](http://drms.readthedocs.io/en/stable/generated/drms.Client.export.html). This creates a 'real' export request, where new FITS files are created at Stanford, containing the image data from the stored files combined with the metadata from the DRMS database.

In [ ]:
request = client.export('hmi.ic_720s[2015.12.19_01:00_TAI]', protocol='fits')
print(request)

In [ ]:
print(request.request_url)

In [ ]:
res = request.download(out_dir)

Open and inspect the new downloaded FITS file:

In [ ]:
fname_ic2 = res.download[0]
f2 = fits.open(fname_ic2)
f2.info()

FITS files from JSOC, that include metadata in their headers are unfortunately not standard-conform. In order to read them with `astropy.io.fits`, it is necessary to fix their headers first.

In [ ]:
f2[1].verify('silentfix')
f2[1].header

In [ ]:
f2[1].data

## Synoptic magnetic map

This example shows how to download synoptic magnetic maps from JSOC and query the metadata separately.

In [ ]:
series = 'hmi.synoptic_mr_720s'
info = client.info(series)

In [ ]:
info.primekeys

In [ ]:
info.segments

In [ ]:
cr = 2150
segment = 'synopMr'

Query the metadata for CR 2150:

In [ ]:
qstr = '{}[{}]'.format(series, cr)

print('Querying "{}"...'.format(qstr))
keys = client.query(qstr, key=drms.const.all)
print('-> {} line(s) retrieved.'.format(len(keys)))

In [ ]:
keys.columns

Perform an `'as-is'` export using only the `'synopMr'` segment:

In [ ]:
expstr = qstr + '{' + segment + '}'

print('Export request: "{}"'.format(expstr))
request = client.export(expstr)
print(request)

In [ ]:
res = request.download(out_dir)

The result is only one record / one file, so we can just use the first row here.

In [ ]:
h = keys.iloc[0]
fname_syn = res.download[0]
#print(h)

Read the image data from the downloaded FITS file.

In [ ]:
data = fits.getdata(fname_syn)
ny, nx = data.shape

Use the metadata from the DRMS query to transform pixel to world coordinates and determine the correct extent and aspect ratio for the plot.

In [ ]:
# Convert pixel to world coordinates using WCS keywords
xmin = (1 - h.CRPIX1)*h.CDELT1 + h.CRVAL1
xmax = (nx - h.CRPIX1)*h.CDELT1 + h.CRVAL1
ymin = (1 - h.CRPIX2)*h.CDELT2 + h.CRVAL2
ymax = (ny - h.CRPIX2)*h.CDELT2 + h.CRVAL2

# Convert to Carrington longitude
xmin = h.LON_LAST - xmin
xmax = h.LON_LAST - xmax

# Compute the plot extent used with imshow
extent = (xmin - abs(h.CDELT1)/2, xmax + abs(h.CDELT1)/2,
          ymin - abs(h.CDELT2)/2, ymax + abs(h.CDELT2)/2)

# Aspect ratio for imshow in respect to the extent computed above
aspect = abs((xmax - xmin)/nx * ny/(ymax - ymin))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 5))

title_str = '{}, Time: {} ... {}'.format(qstr, h.T_START, h.T_STOP)
ax.set_title(title_str, fontsize='medium')

ax.imshow(data, vmin=-300, vmax=300, origin='lower', cmap='gray', extent=extent, aspect=aspect)
ax.invert_xaxis()
ax.set_xlabel('Carrington longitude')
ax.set_ylabel('Sine latitude')

fig.tight_layout()
fig.show()